# Attention Head
* so in transformer we do use multiplicative attention ,there are two type of attention one is additive and other is scale dot attention
* so in research paper  "attention all u need" they hav incorported multiplicative attention
* i assume it just involves addition rather than the multiplication to calculate the score

dimension of data (directly proportional to) variance 

hence the scailing down is down by (dimension of model)**(1/2)


In [ ]:


def attention_head(embeddings,key_weights,query_weights,value_weights,dropout=None,mask=None):
  # first step is to calculate the keys,queries and value vectors for each embeddings
  keys = nn.matmul(embeddings,key_weights)
  queries = nn.matmul(embeddings,query_weights)
  values = nn.matmul(embeddings,value_weights)
  # now calculate the attention scores
  result=[]
  for i, query in enumerate(queries):
    d_k = keys.size(-1)
    # Calculate scores using matrix multiplication between query and transposed keys
    scores = nn.matmul(query, keys.T)
    scores = scores / d_k**0.5
    # replace 0 entries to negative infinite
    if mask is not None:
     scores = scores.masked_fill(mask == 0, -1e9)
    #do log softmax of the whole resulting score
    scores=nn.nn.functional.log_softmax(scores,dim=0)
    # now apply the dropout to ensure that the model weights are not dependent on one portion of data
    if dropout is not None:
        scores = dropout(scores)
    # Apply scores to values using matrix multiplication
    print("scores",scores)
    print("values",values)
    weighted_sum = nn.matmul(scores, values)
    print(f"Weighted sum for query {i}:", weighted_sum)
    result.append(weighted_sum)
  return result




### we will test our attention head with custom args

In [ ]:
import torch.nn as nn_module
import torch

# Define input embeddings and weights for testing with a single embedding
embedding_dim = 4
# Use the existing tensor_3d for testing
embeddings_3d = tensor_3d.float() # Ensure the tensor is float
key_weights = nn.randn(embedding_dim, embedding_dim)
query_weights = nn.randn(embedding_dim, embedding_dim)
value_weights = nn.randn(embedding_dim, embedding_dim)

# Define dropout layer for testing
dropout_layer = nn_module.Dropout(p=0.1) # Example dropout probability of 0.1

# Define a dummy mask for testing (replace with your actual mask logic)
# This dummy mask will mask the second score for each query
mask = torch.ones(embeddings_3d.size(0), embeddings_3d.size(0))
mask[:, 1] = 0 # Example: mask out the second token for all queries


# Test the attention head with the 3 token embeddings and dropout and mask
output = attention_head(embeddings_3d, key_weights, query_weights, value_weights, dropout=dropout_layer, mask=mask)
print("Final output:", output)